In [6]:
import numpy as np
import cv2
import os
import re

In [7]:
#global variables
ssd_results_dictionary = {}
ground_truth_dictionary = {}
iouresults = {}
final_set = set()

treshold = 0.5
path_to_ground_truth = "./TestGround/"
path_to_images = "./TestImages/"
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbikes", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

In [8]:
def intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [9]:
def confusion_matrix(GTdictionary, SSDictionary, category):
    #
    inc = 0
    jnc = 0
    TP = 0
    FP = 0
    FN = 0
    max_check = treshold
    ground_processing_number = 1
    SSD_pricessing_number = 1
    final_set = set()
    
    if not category in SSDictionary:
        FN = int(len(GTdictionary[category])/4)
        TP = 0
        FP = 0
        return TP,FP,FN
    
    while inc < len(GTdictionary[category]):
        #print ("for",ground_processing_number,"we check")
        while jnc < len(SSDictionary[category]):  
            #print("now we check the",SSD_pricessing_number,"SSD result")
            #print("numbers",inc,jnc )
            iou = intersection_over_union(GTdictionary[category][inc:inc+4],SSDictionary[category][jnc:jnc+4])
            print("iou",ground_processing_number,":",SSD_pricessing_number,iou)
            if max_check < iou:
                max_check = iou
                iouresults[ground_processing_number] = SSD_pricessing_number
            jnc += 4
            SSD_pricessing_number += 1 
        #print("The iou check dict",iouresults)
        jnc = 0
        max_check = treshold
        inc += 4
        SSD_pricessing_number = 1
        ground_processing_number += 1
        
    TP = len(iouresults)
    FN = int(len(GTdictionary[category])/4)-len(iouresults)
    for value in iouresults.values():
        final_set.add(value)
    #print("length of original dict:",int(len(SSDictionary[category])/4),"len of values:",len(final_set))
    FP = int(len(SSDictionary[category])/4)-len(final_set)
    iouresults.clear()
    return TP,FP,FN

In [10]:
def get_coordinates_from_ground_truth(file, dictionary):
    groundFile = open(file, "r") #open file
    for line in groundFile:
        if line.find("Original label for object") == 0:
            separated = line.split()
            category_type = re.sub('"','', separated[-1]) #get type of object from txt file
        if line.find("Bounding box for object") == 0:
            separated = line.split()
            #get Top Left, Right Bottom coordinates from file
            coordinates = [int(re.sub('[^0-9]','', separated[12])),int(re.sub('[^0-9]','', separated[13])),int(re.sub('[^0-9]','',separated[15])),int(re.sub('[^0-9]','', separated[16]))]
            #save coordinates in the right dictionary element
            if category_type in dictionary:
                dictionary[category_type] += coordinates
            else:
                dictionary[category_type] = coordinates
    groundFile.close()

In [11]:
print("[INFO] loading model…")
# net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])
# parsing the command from web: 
# python main.py --prototxt MobileNetSSD_deploy.prototxt.txt --model MobileNetSSD_deploy.caffemodel –image car.jpg 
conf_threshold = 0.5
net = cv2.dnn.readNetFromCaffe("MobileNetSSD_deploy.prototxt", "MobileNetSSD_deploy.caffemodel")


[INFO] loading model…


In [13]:
#iterate over images and search for objects
for image_file in os.listdir(path_to_images):
    print (image_file)
    image = cv2.imread(path_to_images+image_file)
    #image = cv2.imread("TestImages/0001.png")
    (h, w) = image.shape[:2]
    #print("Image size",h, w)
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)
    #print("[INFO] computing object detections…")
    net.setInput(blob)
    detections = net.forward()
    for i in np.arange(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
            #print("[INFO] {}".format(label))
            cv2.rectangle(image, (startX, startY), (endX, endY), COLORS[idx], 2)
            
            #print ("type:",CLASSES[idx],"Top left X:Y",startX,":",startY,"Bottom right X:Y",endX,":",endY)
            #y = startY - 15 if startY - 15 > 15 else startY + 15
            if CLASSES[idx] in ssd_results_dictionary:
                ssd_results_dictionary[CLASSES[idx]] += [startX,startY,endX,endY]
            else:
                ssd_results_dictionary[CLASSES[idx]] = [startX,startY,endX,endY]
    
    image_ground_truth = image_file.split(".")[0]+".txt"
    get_coordinates_from_ground_truth(path_to_ground_truth+image_ground_truth,ground_truth_dictionary)
    for x_key in ssd_results_dictionary.keys():
        print("   SSD category:",x_key,"Values:",ssd_results_dictionary[x_key])
      
    for x_key in ground_truth_dictionary.keys():
        print("Ground category:",x_key,"Values:",ground_truth_dictionary[x_key])
    
    for x_cat in ground_truth_dictionary.keys():
        formato = confusion_matrix(ground_truth_dictionary,ssd_results_dictionary,x_cat)
        print("For category '",x_cat,"' True Positive:",formato[0],"False Negative:",formato[2],"False Positive:",formato[1])
        formato = 0
    ground_truth_dictionary.clear()
    ssd_results_dictionary.clear()
            #cv2.putText(image, label, (startX, y),
            #cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
    #cv2.imshow("Output", image)
    #cv2.waitKey(0)
    

person_002.png
   SSD category: person Values: [270, 196, 315, 295]
Ground category: person Values: [265, 195, 317, 299]
iou 1 : 1 0.8237795857988166
For category ' person ' True Positive: 1 False Negative: 0 False Positive: 0
person_018.png
Ground category: person Values: [61, 66, 83, 111, 133, 33, 217, 255, 209, 36, 268, 256]
For category ' person ' True Positive: 0 False Negative: 3 False Positive: 0
person_013.png
   SSD category: person Values: [78, 74, 158, 272]
Ground category: person Values: [81, 68, 163, 277]
iou 1 : 1 0.8598014888337469
For category ' person ' True Positive: 1 False Negative: 0 False Positive: 0
person_008.png
   SSD category: person Values: [197, 12, 286, 350]
Ground category: person Values: [197, 33, 289, 339]
iou 1 : 1 0.8785161290322581
For category ' person ' True Positive: 1 False Negative: 0 False Positive: 0
person_009.png
   SSD category: person Values: [152, 158, 191, 271, 93, 157, 134, 272]
Ground category: person Values: [1, 171, 39, 274, 92, 161,